# Deploying functions 

In [132]:
#Reference
#https://docs.openfaas.com/tutorials/kubernetes-hpa/

In [133]:
#Deploy function and enable the function auto scaling by openfaas
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f aeneas.yml --annotation topic="aeneas" --env max_inflight=50 --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=5 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: aeneas.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/aeneas



In [134]:
#Deploy function and enable the function auto scaling by openfaas
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f scaling-aeneas-tocloud.yml --annotation topic="tocloud" --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=50 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: scaling-aeneas-tocloud.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/scaling-aeneas-tocloud



In [135]:
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f getobject.yml  --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=50 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: getobject.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/getobject



In [136]:
!ssh ubuntu@172.17.141.197 "cd scalingsdp;faas deploy  -f pubrabbitmq.yml  --label com.openfaas.scale.min=1 --label com.openfaas.scale.max=1 --label com.openfaas.scale.target=50 --label com.openfaas.scale.type=rps --label com.openfaas.scale.target-proportion=0.90;"

Deploying: pubrabbitmq.

Deployed. 202 Accepted.
URL: http://127.0.0.1:8080/function/pubrabbitmq



In [137]:
!ssh ubuntu@172.17.141.197 "faas describe scaling-aeneas-mqtt"

No such function: scaling-aeneas-mqtt


# Deploying Rabbitmq Connector 

In [138]:
# Deploying connector configurartion
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl apply -f cf.yaml"

configmap/rabbitmq-connector-configmap unchanged


In [139]:
# Deploy connector
!ssh ubuntu@172.17.141.197 "cd rabbitmq-connector/artifacts;sudo kubectl apply  -f configmap.yaml"

deployment.apps/rabbitmq-connector configured


In [140]:
!ssh ubuntu@172.17.141.197 "sudo kubectl get po -n openfaas" 

NAME                                  READY   STATUS    RESTARTS         AGE
nats-5b77986b8-48p2x                  1/1     Running   4 (5d21h ago)    42d
queue-worker-7d8668444f-tmq7b         1/1     Running   5 (5d21h ago)    27d
alertmanager-6cd7bd5c64-hrcrt         1/1     Running   4 (5d21h ago)    27d
basic-auth-plugin-78b958969c-tnpck    1/1     Running   4 (5d21h ago)    27d
gateway-bf4f5d6f7-4q7kn               2/2     Running   10 (4d16h ago)   27d
prometheus-7678c644d6-k28wr           1/1     Running   0                72m
rabbitmq-connector-64cbd548f5-vtw26   1/1     Running   0                69m


# Deploying KEDA and components 

In [141]:
#Install KEDA 
!ssh ubuntu@172.17.141.197 "sudo kubectl apply -f https://github.com/kedacore/keda/releases/download/v2.9.0/keda-2.9.0.yaml"

namespace/keda unchanged
customresourcedefinition.apiextensions.k8s.io/clustertriggerauthentications.keda.sh unchanged
customresourcedefinition.apiextensions.k8s.io/scaledjobs.keda.sh unchanged
customresourcedefinition.apiextensions.k8s.io/scaledobjects.keda.sh unchanged
customresourcedefinition.apiextensions.k8s.io/triggerauthentications.keda.sh unchanged
serviceaccount/keda-operator unchanged
clusterrole.rbac.authorization.k8s.io/keda-external-metrics-reader unchanged
clusterrole.rbac.authorization.k8s.io/keda-operator configured
rolebinding.rbac.authorization.k8s.io/keda-auth-reader unchanged
clusterrolebinding.rbac.authorization.k8s.io/keda-hpa-controller-external-metrics unchanged
clusterrolebinding.rbac.authorization.k8s.io/keda-operator unchanged
clusterrolebinding.rbac.authorization.k8s.io/keda-system-auth-delegator unchanged
service/keda-metrics-apiserver unchanged
service/keda-operator unchanged
deployment.apps/keda-metrics-apiserver configured
deployment.apps/keda-operator c

In [142]:
#check for the keda components
!ssh ubuntu@172.17.141.197 "sudo kubectl get po -n keda"

NAME                                      READY   STATUS    RESTARTS           AGE
keda-metrics-apiserver-5b4c56c8c8-k7f2s   1/1     Running   4 (5d21h ago)      26d
keda-operator-598b95d667-xvd8b            1/1     Running   2191 (4d16h ago)   26d


In [143]:
#Create Scaler
!ssh ubuntu@172.17.141.197 "cd /home/ubuntu/rabbitmq-testing/sample-go-rabbitmq/deploy;sudo kubectl apply -f deploy-consumer-aeneas.yaml"


secret/rabbitmq-consumer-secret unchanged
scaledobject.keda.sh/rabbitmq-aeneas unchanged
triggerauthentication.keda.sh/rabbitmq-consumer-trigger unchanged


# Experiments 

In [144]:
import sys
import  datetime
import os
from minio import Minio
import csv
import json
import pandas as pd
import time
import subprocess
from subprocess import Popen, PIPE
import requests
from IPython.display import display

In [145]:
scenario = "keda"
user=10
iteration=0.07
minio_host="172.17.141.197:9001"
bucket_name="aeneas-output"
PROMETHEUS = 'http://172.17.141.197:31022/'
time_locust=120


In [146]:
!rm input_data.csv
!touch input_data.csv
!echo "intime" > input_data.csv

In [147]:
!locust --headless -f testingrabbitlocust.py --host='127.0.0.1' --skip-log -t 120s  --users 10 --spawn-rate 0.07

In [148]:
time.sleep(80)

In [149]:
df= pd.read_csv('input_data.csv')
print(df)

                          intime
0   '2023-01-17 11:47:06.424827'
1   '2023-01-17 11:47:13.760174'
2   '2023-01-17 11:47:22.096899'
3   '2023-01-17 11:47:26.423462'
4   '2023-01-17 11:47:35.183115'
5   '2023-01-17 11:47:40.308235'
6   '2023-01-17 11:47:46.416692'
7   '2023-01-17 11:47:46.732450'
8   '2023-01-17 11:47:52.734239'
9   '2023-01-17 11:47:57.167399'
10  '2023-01-17 11:47:59.020418'
11  '2023-01-17 11:48:03.906033'
12  '2023-01-17 11:48:05.844673'
13  '2023-01-17 11:48:06.418568'
14  '2023-01-17 11:48:06.483706'
15  '2023-01-17 11:48:12.379476'
16  '2023-01-17 11:48:12.862812'
17  '2023-01-17 11:48:13.869056'
18  '2023-01-17 11:48:20.447601'
19  '2023-01-17 11:48:20.622212'
20  '2023-01-17 11:48:21.040010'
21  '2023-01-17 11:48:22.299690'
22  '2023-01-17 11:48:26.417905'
23  '2023-01-17 11:48:29.572391'
24  '2023-01-17 11:48:30.200639'
25  '2023-01-17 11:48:30.301937'
26  '2023-01-17 11:48:33.427275'
27  '2023-01-17 11:48:39.278217'
28  '2023-01-17 11:48:40.654545'
29  '2023-

In [150]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    outtime.append(obj.last_modified)#
    client.remove_object(bucket_name, obj.object_name)

In [151]:
df['outtime']= outtime
df['outtime']= pd.to_datetime(df['outtime'],errors='coerce').dt.tz_convert(None)
df['intime']= pd.to_datetime(df['intime'],errors='coerce')
df =df.sort_values(by="outtime")
display(df)
print(df['outtime'].iloc[-1]-df['intime'].iloc[0])

,intime,outtime
0,2023-01-17 11:47:06.424827,2023-01-17 11:47:14.673
1,2023-01-17 11:47:13.760174,2023-01-17 11:47:18.788
2,2023-01-17 11:47:22.096899,2023-01-17 11:47:26.432
3,2023-01-17 11:47:26.423462,2023-01-17 11:47:30.792
4,2023-01-17 11:47:35.183115,2023-01-17 11:47:39.572
5,2023-01-17 11:47:40.308235,2023-01-17 11:47:44.664
6,2023-01-17 11:47:46.416692,2023-01-17 11:47:53.111
7,2023-01-17 11:47:46.732450,2023-01-17 11:47:53.303
8,2023-01-17 11:47:52.734239,2023-01-17 11:47:57.464
9,2023-01-17 11:47:57.167399,2023-01-17 11:48:02.506


0 days 00:02:21.320173


In [152]:
def get_function_execution_time(cmd):
    with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
        return (process.communicate()[0].decode("utf-8"))

In [153]:
#cmd1 = "sudo kubectl logs gateway-7b8d9dbb5b-rhnwl  -n openfaas -c gateway | grep getobject | cut -c 71-77 | tail -"
cmd1="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn  -n openfaas -c gateway  | grep /function/aeneas | grep 200 | cut -c 80-83 | tail -"+str(len(outtime))
cmd2="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/scaling-aeneas-tocloud | cut -c 96-101 | tail -"+str(len(outtime))
cmd3="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/getobject | grep POST | cut -c 71-76  | tail -"+str(len(outtime))
cmd4="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-bf4f5d6f7-4q7kn -n openfaas -c gateway  | grep /function/pubrabbitmq | grep POST | cut -c 73-78 | tail -"+str(len(outtime))
df['aeneas']=(get_function_execution_time(cmd1)).split("\n")[:-1]
#df['aeneas']=(pd.to_timedelta(df['aeneas'].astype(float), unit='s'))

df['tocloud']=(get_function_execution_time(cmd2)).split("\n")[:-1]
#df['tocloud']=(pd.to_timedelta(df['tocloud'].astype(float), unit='s'))

#df['getobject']=(get_function_execution_time(cmd3)).split("\n")[:-1]
df['pubrabbitmq']=((get_function_execution_time(cmd4)).split("\n")[:-1])
#df['pubrabbitmq']=(pd.to_timedelta(df['pubrabbitmq'].astype(float), unit='s'))

df

,intime,outtime,aeneas,tocloud,pubrabbitmq
0,2023-01-17 11:47:06.424827,2023-01-17 11:47:14.673,4.38,0.0452,0.0394
1,2023-01-17 11:47:13.760174,2023-01-17 11:47:18.788,4.92,0.0469,0.0351
2,2023-01-17 11:47:22.096899,2023-01-17 11:47:26.432,4.23,0.0434,0.0362
3,2023-01-17 11:47:26.423462,2023-01-17 11:47:30.792,4.27,0.0468,0.0379
4,2023-01-17 11:47:35.183115,2023-01-17 11:47:39.572,4.29,0.0429,0.0474
5,2023-01-17 11:47:40.308235,2023-01-17 11:47:44.664,4.25,0.0396,0.0388
6,2023-01-17 11:47:46.416692,2023-01-17 11:47:53.111,6.66,0.0456,0.0427
7,2023-01-17 11:47:46.732450,2023-01-17 11:47:53.303,6.48,0.0399,0.0354
8,2023-01-17 11:47:52.734239,2023-01-17 11:47:57.464,4.63,0.0356,0.0366
9,2023-01-17 11:47:57.167399,2023-01-17 11:48:02.506,5.23,0.0526,0.0432


In [154]:
#df =df.sort_values(by="outtime")
df['TPT']=(df['outtime']-df['intime'])
df['TPT']=df['TPT'].values.astype('datetime64[ns]')
df['TPT']= pd.to_datetime(df['TPT']).dt.strftime('%S.%f')
df['TPT']=df['TPT'].astype(float)
df['PT']= df['TPT'].round(decimals = 0)
df['FET'] = (df['aeneas']).astype(float) + (df['tocloud']).astype(float)
#df['MQ time']= ((df['TPT']).astype(float) - df['FET'])
df["scenario"]= ""+str(user)+"_"+str(iteration)+"_"+scenario
display(df)
df.to_csv(""+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv')

,intime,outtime,aeneas,tocloud,pubrabbitmq,TPT,PT,FET,scenario
0,2023-01-17 11:47:06.424827,2023-01-17 11:47:14.673,4.38,0.0452,0.0394,8.248173,8.0,4.4252,10_0.05_keda
1,2023-01-17 11:47:13.760174,2023-01-17 11:47:18.788,4.92,0.0469,0.0351,5.027826,5.0,4.9669,10_0.05_keda
2,2023-01-17 11:47:22.096899,2023-01-17 11:47:26.432,4.23,0.0434,0.0362,4.335101,4.0,4.2734,10_0.05_keda
3,2023-01-17 11:47:26.423462,2023-01-17 11:47:30.792,4.27,0.0468,0.0379,4.368538,4.0,4.3168,10_0.05_keda
4,2023-01-17 11:47:35.183115,2023-01-17 11:47:39.572,4.29,0.0429,0.0474,4.388885,4.0,4.3329,10_0.05_keda
5,2023-01-17 11:47:40.308235,2023-01-17 11:47:44.664,4.25,0.0396,0.0388,4.355765,4.0,4.2896,10_0.05_keda
6,2023-01-17 11:47:46.416692,2023-01-17 11:47:53.111,6.66,0.0456,0.0427,6.694308,7.0,6.7056,10_0.05_keda
7,2023-01-17 11:47:46.732450,2023-01-17 11:47:53.303,6.48,0.0399,0.0354,6.570550,7.0,6.5199,10_0.05_keda
8,2023-01-17 11:47:52.734239,2023-01-17 11:47:57.464,4.63,0.0356,0.0366,4.729761,5.0,4.6656,10_0.05_keda
9,2023-01-17 11:47:57.167399,2023-01-17 11:48:02.506,5.23,0.0526,0.0432,5.338601,5.0,5.2826,10_0.05_keda


In [155]:
pre_url = PROMETHEUS + '/api/v1/query_range?query='
timestamp= pd.to_datetime(df['outtime'].iloc[-1], format='%Y-%m-%d %H:%M:%S')
time_interval = (time.mktime(timestamp.timetuple()))
user=df.shape[0]
end=(time.mktime(timestamp.timetuple()))
start=pd.to_datetime(df['intime'].iloc[0], format='%Y-%m-%d %H:%M:%S')
start=(time.mktime(start.timetuple()))
print(start,end)

1673956026.0 1673956167.0


In [156]:
interval = (df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()
interval= round(interval)

In [157]:
interval= round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) if round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) > 60  else 60
print(interval)

141


In [158]:
import copy 
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
    #data2=res.get('data').get('result')[0].get('values')
    data_dict={}
    metric_list = []
    # print(data['data']['result']['values'])
    # exit()
    for i in res['data']['result']:
        for j in i['values']:
            data_dict = copy.deepcopy(i['metric'])
            data_dict['time'] = j[0]
            data_dict['value'] = j[1]
            metric_list.append(data_dict)
    df_metric = pd.DataFrame(metric_list)
    return df_metric

In [159]:
user=df.shape[0]
metrics = {'workload_type':""+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration),'workload_size':user,'Workload TPT':(df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds(),'Average Workload TPT':df['TPT'].mean(),'Average FET':df['FET'].mean()}

promo_metrics = pd.DataFrame(columns=['cpu_used','cpu_requested'])

In [160]:

expr_namespace_cpu_requested = 'sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas-fn", resource="cpu"})&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['cpu_requested']=getdataprometheus(pre_url+expr_namespace_cpu_requested)['value']

promo_metrics['time']=getdataprometheus(pre_url+expr_namespace_cpu_requested)['time']
expr_namespace_cpu_used = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['cpu_used']=getdataprometheus(pre_url+expr_namespace_cpu_used)['value']

expr_namespace_pod_count = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn"})&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['pod_count']=getdataprometheus(pre_url+expr_namespace_pod_count)['value']

expr_namespace_memory_used = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['memory_used']=getdataprometheus(pre_url+expr_namespace_memory_used)['value']

expr_namespace_memory_requested = 'sum(rate(container_memory_working_set_bytes{container_name!="POD",namespace="openfaas-fn"}[60s]))&start='+str(start)+'&end='+str(end)+'&step=10s'
promo_metrics['memory_used']=getdataprometheus(pre_url+expr_namespace_memory_requested)['value']

promo_metrics['time']= pd.to_datetime(promo_metrics['time'],unit='s')
seconds = []
for i in range(promo_metrics.shape[0]): 
    seconds.append(i*10)
promo_metrics['seconds'] = seconds

throughput_qurty = 'sum (rate(gateway_function_invocation_total{code="200"}[10s]))'
promo_metrics['throughput']= getdataprometheus('http://172.17.141.197:31376/api/v1/query_range?query=sum (rate(gateway_function_invocation_total{code="200"}[20s]))&start='+str(start)+'&end='+str(end)+'&step=10s')['value']
promo_metrics.to_csv("keda_results/promo_metrics"+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv',index=False)


# #expr_namespace_cpu_requested = 'sum(node_namespace_pod_container:container_cpu_usage_seconds_total:sum_irate{namespace="openfaas-fn"})&time='+str(time_interval)
# expr_namespace_cpu_used='sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_used']=getdataprometheus(pre_url+expr_namespace_cpu_used)
# expr_namespace_cpu_requested='sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas-fn", resource="cpu"})&time='+str(time_interval)
# metrics['CPU_requested']=getdataprometheus(pre_url+expr_namespace_cpu_requested)

# expr_pod_cpu_rabbitmq_connector = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# expr_pod_memory_rabbitmq_connector = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_rabbitmq_broker_used']=getdataprometheus(pre_url+expr_pod_cpu_rabbitmq_connector)
# metrics['Memory_rabbitmq_broker_used']=getdataprometheus(pre_url+expr_pod_memory_rabbitmq_connector)

# expr_pod_cpu_rabbitmq_connector_requested = 'sum(kube_pod_container_resource_requests{job="kube-state-metrics", namespace="openfaas", resource="cpu",pod=~"rabbitmq.*"})&time='+str(time_interval)
# expr_pod_memory_rabbitmq_connector = 'sum(rate(container_memory_usage_bytes{container_name!="POD",namespace="openfaas",pod=~"rabbitmq.*"}['+str(interval)+'s]))&time='+str(time_interval)
# metrics['CPU_rabbitmq_broker_requested']=getdataprometheus(pre_url+expr_pod_cpu_rabbitmq_connector_requested)

# expr_pod_rabbitmq_function = 'sum(kube_pod_container_status_ready{namespace="openfaas",pod=~"rabbitmq.*"})&time='+str(time_interval)
# metrics['pod_count_rabbitmq']=getdataprometheus(pre_url+expr_pod_rabbitmq_function)

# expr_pod_aeneas_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"aeneas.*"})&time='+str(time_interval)
# expr_pod_tocloud_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"scaling-aeneas-tocloud.*"})&time='+str(time_interval)
# expr_pod_getobject_function = 'sum(kube_pod_container_status_ready{namespace="openfaas-fn",pod=~"getobject.*"})&time='+str(time_interval)
# print(pre_url+expr_pod_aeneas_function)
# metrics['pod_count_aeneas']=getdataprometheus(pre_url+expr_pod_aeneas_function)
# metrics['pod_count_tocloud']=getdataprometheus(pre_url+expr_pod_tocloud_function)
# metrics['pod_count_getobject']=getdataprometheus(pre_url+expr_pod_getobject_function)

# metrics

In [161]:
df_metrics = pd.read_csv("overall_keda.csv")
df_metrics = df_metrics.append(metrics, ignore_index=True)
display(df_metrics)

/tmp/ipykernel_17734/4126905209.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metrics = df_metrics.append(metrics, ignore_index=True)


,Average Workload TPT,Workload TPT,workload_size,workload_type,Average FET
0,11.965319,143.776106,45,45_120_keda,11.506742
1,8.975417,128.887709,47,47_120_keda,8.582521
2,8.472999,123.443820,44,44_120_keda,8.069307
3,13.348710,141.320173,47,47_120_keda_0.05,12.668977


In [162]:
df_metrics.to_csv("overall_keda.csv",index=False)
display(df_metrics)

,Average Workload TPT,Workload TPT,workload_size,workload_type,Average FET
0,11.965319,143.776106,45,45_120_keda,11.506742
1,8.975417,128.887709,47,47_120_keda,8.582521
2,8.472999,123.443820,44,44_120_keda,8.069307
3,13.348710,141.320173,47,47_120_keda_0.05,12.668977


In [163]:
df['PT'].describe()
df['PT'].quantile([0.0, .9])

0.0     4.0
0.9    21.0
Name: PT, dtype: float64

In [164]:
# Frequency
stats_df = df \
.groupby('PT') \
['PT'] \
.agg('count') \
.pipe(pd.DataFrame) \
.rename(columns = {'PT': 'frequency'})

# PDF
stats_df['pdf'] = stats_df['frequency'] / sum(stats_df['frequency'])
stats_df['cdf'] = stats_df['pdf'].cumsum()
stats_df = stats_df.reset_index()
display(stats_df)
stats_df.to_csv("stats_df_"+str(user)+"_"+str(time_locust)+"_"+scenario+"_"+str(iteration)+'.csv')
# from matplotlib import pyplot as plt

# plt.plot(stats_df['PT'],stats_df['cdf'],  linestyle='--', marker='o', color='b', label='k8s_k8s')
# #plt.plot(df['user'],df['CT'],  linestyle='--', marker='*', color='r', label='Communication Time')
# plt.grid()
# plt.legend(loc='best')
# plt.xlabel('PT (Seconds)', fontsize=18)
# plt.ylabel('CDF', fontsize=16)


,PT,frequency,pdf,cdf
0,4.0,4,0.085106,0.085106
1,5.0,3,0.063830,0.148936
2,6.0,3,0.063830,0.212766
3,7.0,2,0.042553,0.255319
4,8.0,2,0.042553,0.297872
5,11.0,1,0.021277,0.319149
6,12.0,7,0.148936,0.468085
7,13.0,2,0.042553,0.510638
8,14.0,2,0.042553,0.553191
9,15.0,3,0.063830,0.617021


In [165]:
!ssh ubuntu@172.17.141.197 "cd /home/ubuntu/rabbitmq-testing/sample-go-rabbitmq/deploy;sudo kubectl delete -f deploy-consumer-aeneas.yaml -n openfaas-fn"


secret "rabbitmq-consumer-secret" deleted
scaledobject.keda.sh "rabbitmq-aeneas" deleted
triggerauthentication.keda.sh "rabbitmq-consumer-trigger" deleted
